In [45]:
import requests
import ast # abstract syntax tree
import pandas as pd
from bs4 import BeautifulSoup

In [52]:
# get brewery names and locations
URL= 'https://www.brewbound.com/breweries'

soup = BeautifulSoup(requests.get(URL).content, "html.parser")

data = soup.find_all('script')[-4]
string_results = str(data).split("\r\n")[2]
string_results_list = "["+string_results[:len(string_results) - 7]+"]"
results = ast.literal_eval(string_results_list)
df = pd.DataFrame(results, columns = ['brewery_name', 'latitude','longitude','url'])

df.head()

,brewery_name,latitude,longitude,url
0,Faust Hotel Restaurant and Brew PUB,29.701406,-98.123061,/breweries/Faust_Hotel_Restaurant_and_Brew_PUB
1,Wolf Pack Brewing Company,44.662053,-111.099461,/breweries/Wolf_Pack_Brewing_Co
2,Defiant Brewing Company,41.058405,-74.022847,/breweries/Defiant_Brewing_Co
3,El Toro Brewing Company,37.146552,-121.621987,/breweries/El_Toro_Brewing_Co
4,Sebago Brewing Company,43.679212,-70.396424,/breweries/Sebago_Brewing_Co


In [57]:
# use brewery url to get beers
URL = "https://www.brewbound.com/breweries/21st_amendment_brewery"

# soup = BeautifulSoup(requests.get(URL).content, "html.parser")
# data = soup.find_all('script')[-4]
page = requests.get(URL)

# print(page.text)


soup = BeautifulSoup(page.content, "html.parser")
results = soup.find(id="beers")

results

<div class="tab-content brewery-inventory" id="beers">
<div style="background-color: #fff; padding-left: 20px; padding-right: 20px;">
<br/>
<div style="float: right;">
<a href="/breweries/21st_Amendment_Brewery/RequestEditAccess" style="font-weight: bold !important">Request access to edit this brewery listing</a> </div>
<h2>Year Round Beers</h2>
<table class="featuredjobs">
<colgroup>
<col/>
<col/>
<col width="25%"/>
</colgroup>
<thead>
<tr>
<th colspan="2" scope="col">Beer Name</th>
<th scope="col">Style</th>
</tr>
</thead>
<tbody>
<tr>
<td width="90px">
<figure>
<a href="/breweries/21st_Amendment_Brewery/down-to-earth">
<div class="center-cropped" style="width: 70px; background-image: url('https://d2pxm94gkd1wuq.cloudfront.net/Beers/SquareBig/5181244.21st.downtoearth.jpg');">
</div>
</a>
</figure>
</td>
<td><a href="/breweries/21st_Amendment_Brewery/down-to-earth"><strong>Down to Earth</strong></a></td>
<td>Session Beer</td>
</tr>
<tr>
<td width="90px">
<figure>
<a href="/breweries/2

In [ ]:
# use beer url to get beer info